In [1]:
import numpy as np
import sympy
from sympy.tensor.tensor import TensorIndexType, TensorHead, tensor_indices
from tqdm import tqdm

sympy.init_printing()

In [2]:
A, B, C, D, E, F, G, H, I = sympy.symbols("A B C D E F G H I")
F_01, F_02, F_03, F_12, F_13, F_23 = sympy.symbols("F_{01} F_{02} F_{03} F_{12} F_{13} F_{23}")

In [3]:
g_mat = sympy.Matrix([
    [A + sympy.I * B, C + sympy.I * D, 0, 0],
    [C + sympy.I * D, E + sympy.I * F, 0, 0],
    [0, 0, G + sympy.I * H, 0],
    [0, 0, 0, I]
])
g_mat

⎡A + ⅈ⋅B  C + ⅈ⋅D     0     0⎤
⎢                            ⎥
⎢C + ⅈ⋅D  E + ⅈ⋅F     0     0⎥
⎢                            ⎥
⎢   0        0     G + ⅈ⋅H  0⎥
⎢                            ⎥
⎣   0        0        0     I⎦

In [4]:
F_mat = sympy.Matrix([
    [0, F_01, F_02, F_03],
    [-F_01, 0, F_12, F_13],
    [-F_02, -F_12, 0, F_23],
    [-F_03, -F_13, -F_23, 0]
])
F_mat

⎡   0     F_{01}   F_{02}   F_{03}⎤
⎢                                 ⎥
⎢-F_{01}     0     F_{12}   F_{13}⎥
⎢                                 ⎥
⎢-F_{02}  -F_{12}     0     F_{23}⎥
⎢                                 ⎥
⎣-F_{03}  -F_{13}  -F_{23}    0   ⎦

In [5]:
Lorentzian = TensorIndexType('Lorentzian', dummy_name='L')
i0, i1, i2, i3 = tensor_indices('i0:4', Lorentzian)
j0, j1, j2, j3 = tensor_indices('j0:4', Lorentzian)

In [6]:
ggFF_mat = sympy.tensorproduct(sympy.tensorproduct(g_mat, g_mat), sympy.tensorproduct(F_mat, F_mat))

In [7]:
ggFF_tensor = TensorHead('ggFF', [Lorentzian] * 8)
repl = {ggFF_tensor(i0, j0, i1, j1, -i2, -j2, -i3, -j3): ggFF_mat}

Compute the quadratic form.

In [8]:
quadratic_form = ggFF_tensor(i0, j0, i1, j1, -i0, -i1, -j0, -j1).replace_with_arrays(repl)
quadratic_form

In [9]:
quadratic_form = quadratic_form.expand().simplify().collect(sympy.I)
quadratic_form

Take the real part.

In [10]:
real_quadratic_form = quadratic_form - quadratic_form.coeff(sympy.I) * sympy.I
real_quadratic_form

In [11]:
real_quadratic_form.factor().collect([F_01, F_02, F_03, F_12, F_13, F_23])

Compute the real part when the metric is real.

In [12]:
real_quadratic_form_g_real = real_quadratic_form.subs({B: 0, D: 0, F: 0, H: 0}).collect([F_01, F_02, F_03, F_12, F_13, F_23])
real_quadratic_form_g_real

#### $g$ Inverse

In [13]:
g_mat_inv = g_mat.inv()
g_mat_inv

⎡                   E + ⅈ⋅F                                        -C - ⅈ⋅D   
⎢─────────────────────────────────────────────  ──────────────────────────────
⎢                             2              2                               2
⎢A⋅E + ⅈ⋅A⋅F + ⅈ⋅B⋅E - B⋅F - C  - 2⋅ⅈ⋅C⋅D + D   A⋅E + ⅈ⋅A⋅F + ⅈ⋅B⋅E - B⋅F - C 
⎢                                                                             
⎢                   -C - ⅈ⋅D                                       A + ⅈ⋅B    
⎢─────────────────────────────────────────────  ──────────────────────────────
⎢                             2              2                               2
⎢A⋅E + ⅈ⋅A⋅F + ⅈ⋅B⋅E - B⋅F - C  - 2⋅ⅈ⋅C⋅D + D   A⋅E + ⅈ⋅A⋅F + ⅈ⋅B⋅E - B⋅F - C 
⎢                                                                             
⎢                                                                             
⎢                      0                                              0       
⎢                                                   

In [14]:
k = sympy.fraction(g_mat_inv[0, 0])[1].collect(sympy.I)
k

In [16]:
k_star = A * E - B * F - C ** 2 + D ** 2 - sympy.I * k.coeff(sympy.I)
k_star

In [17]:
kappa = sympy.symbols("kappa")
kappa_val = (k * k_star).expand().simplify().factor()
kappa_val

In [18]:
delta = sympy.symbols("delta")
delta_val = G ** 2 + H ** 2
delta_val

These are the matrix elements of the inverse.

In [19]:
A_inv = (A * (E ** 2 + F ** 2) + E * (D ** 2 - C ** 2) - 2 * C * D * F) / kappa_val
B_inv = (F * (D ** 2 - C ** 2) - B * (E ** 2 + F ** 2) + 2 * C * D * E) / kappa_val
C_inv = (B * (C * F - D * E) - A * (C * E + D * F) + C * (C ** 2 + D ** 2)) / kappa_val
D_inv = (A * (C * F - D * E) + B * (C * E + D * F) - D * (C ** 2 + D ** 2)) / kappa_val
E_inv = (E * (A ** 2 + B ** 2) + A * (D ** 2 - C ** 2) - 2 * B * C * D) / kappa_val
F_inv = (B * (D ** 2 - C ** 2) - F * (A ** 2 + B ** 2) + 2 * A * C * D) / kappa_val
G_inv = G / delta_val
H_inv = -H / delta_val
I_inv = 1 / I

#### Sample matrices

It is difficult to prove analytically that the conditions for $g^{\mu\nu}$ are the same as the conditions for $g_{\mu\nu}$. However, it can be validated numerically by sampling matrices that fulfill the conditions, inverting them, then checking whether they also satisfy the conditions.

In [23]:
def _is_q_2_satisfied(A_val, B_val, C_val, D_val, E_val, F_val, G_val, H_val, I_val):
    """Determine whether the q = 2 conditions are satisfied for the given matrix elements."""
    if A_val <= 0 or E_val <= 0 or G_val <= 0 or I_val <= 0:
        return False

    cond1 = A_val * E_val - C_val ** 2
    if cond1 <= 0:
        return False

    cond2 = A_val * E_val - C_val ** 2 - B_val * F_val + D_val ** 2
    if cond2 <= 0:
        return False

    cond3 = A_val * G_val - B_val * H_val
    if cond3 <= 0:
        return False

    cond4 = E_val * G_val - F_val * H_val
    if cond4 <= 0:
        return False

    cond5 = cond3 * cond4 - (C_val * G_val - D_val * H_val) ** 2
    if cond5 <= 0:
        return False

    return True

In [24]:
num_samples = int(1e6)
max_num_range = 1e6

In [25]:
non_q_2_inv_matrices = []
num_q_2_inv_matrices = 0
total_q_2_matrices = 0

In [26]:
for i in tqdm(range(num_samples)):
    # A, E, G, I > 0
    A_val, E_val, G_val, I_val = np.random.uniform(0, max_num_range, size=4)

    # sample imaginary parts
    B_val, C_val, D_val, F_val, H_val = np.random.uniform(-max_num_range, max_num_range, size=5)

    # only proceed if the matrix satisfies the q = 2 conditions
    if not _is_q_2_satisfied(A_val, B_val, C_val, D_val, E_val, F_val, G_val, H_val, I_val):
        continue

    vals_dict = {
        A: A_val, B: B_val, C: C_val, D: D_val, E: E_val, F: F_val, G: G_val, H: H_val, I: I_val
    }

    # compute the matrix elements of the inverse
    A_val_inv = A_inv.subs(vals_dict)
    B_val_inv = B_inv.subs(vals_dict)
    C_val_inv = C_inv.subs(vals_dict)
    D_val_inv = D_inv.subs(vals_dict)
    E_val_inv = E_inv.subs(vals_dict)
    F_val_inv = F_inv.subs(vals_dict)
    G_val_inv = G_inv.subs(vals_dict)
    H_val_inv = H_inv.subs(vals_dict)
    I_val_inv = I_inv.subs(vals_dict)

    if not _is_q_2_satisfied(A_val_inv, B_val_inv, C_val_inv, D_val_inv, E_val_inv, F_val_inv, G_val_inv, H_val_inv, I_val_inv):
        # log matrix for which the inverse does not satisfy q = 2 conditions
        non_q_2_inv_matrices.append(vals_dict)
    else:
        # increment number of q = 2 inverse matrices
        num_q_2_inv_matrices += 1

    # increment number of q = 2 sampled
    total_q_2_matrices += 1

100%|██████████| 1000000/1000000 [2:33:59<00:00, 108.23it/s]  


In [27]:
len(non_q_2_inv_matrices)

In [28]:
num_q_2_inv_matrices

In [29]:
total_q_2_matrices